# Dummy Net - Revisited with Gradients

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import tensorflow as tf
import numpy as np
import os
import shutil
from utils.data import init_dir
from utils.nn_visualization import variable_summaries

## Build Net Graph

In [4]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('dummy_net_inputs'):
        data = tf.placeholder(tf.float32, shape=[None, 3], name='data')
        labels = tf.placeholder(tf.float32, shape=[None, 1], name='labels')
    
    with tf.variable_scope('layer_1'):
        w = tf.get_variable('w', initializer=tf.constant_initializer(0), shape=[3,2])
        prediction = tf.matmul(data, w)
    
    with tf.variable_scope('layer_2'):
        w = tf.get_variable('w', initializer=tf.constant_initializer(0.1), shape=[2,1])
        prediction = tf.matmul(prediction, w)
    
    with tf.name_scope('loss'):
        loss = tf.losses.mean_squared_error(labels, prediction)
        tf.summary.scalar('loss_summary', loss)
    
    with tf.name_scope('training'):
        all_trainable_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        
        all_trainable_vars_names = list(map(lambda var: var.name, all_trainable_vars))
        print("Trainable variables: ", all_trainable_vars_names)
        
        gradients = tf.gradients(loss, all_trainable_vars)
        variable_summaries('summary_gradient_w1', gradients[0])
        
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
        train_step = optimizer.apply_gradients(zip(gradients, all_trainable_vars))

    initialize_vars = tf.global_variables_initializer()
    merge_summaries = tf.summary.merge_all()

Trainable variables:  ['layer_1/w:0', 'layer_2/w:0']


## Init Model Logging

In [5]:
from utils.data import init_model_logging
base_dir = '/tensorboard_summaries/dummy_net/'
exp_name = 'experiment_revisited'

logging_meta = init_model_logging(base_dir, exp_name, graph=graph, remove_existing=True)

## Run Net

In [6]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True

_data = np.array([[1,2,3]])
_labels = np.array([[5]])

with tf.Session(graph=graph, config=config) as session:
    session.run(initialize_vars)
    
    for iteration in range(1000):
        feed_dict = {
            data: _data, 
            labels: _labels
        }
        
        _, _prediction, _loss, _summary = session.run([train_step, prediction, loss, merge_summaries],
                            feed_dict=feed_dict)
        
        logging_meta['train_writer'].add_summary(_summary, iteration)
        print("Iteration {}: loss {}, pred {}".format(iteration, _loss, _prediction[0][0]))
    logging_meta['saver'].save(session, model_path)

Iteration 0: loss 25.0, pred 0.0
Iteration 1: loss 24.860197067260742, pred 0.013999998569488525
Iteration 2: loss 24.711469650268555, pred 0.028936689719557762
Iteration 3: loss 24.5343017578125, pred 0.04678851738572121
Iteration 4: loss 24.307300567626953, pred 0.06975671648979187
Iteration 5: loss 24.0048885345459, pred 0.10052148252725601
Iteration 6: loss 23.59521484375, pred 0.14250966906547546
Iteration 7: loss 23.038270950317383, pred 0.2001798152923584
Iteration 8: loss 22.284957885742188, pred 0.2793055474758148
Iteration 9: loss 21.277956008911133, pred 0.3871963322162628
Iteration 10: loss 19.956560134887695, pred 0.5327236652374268
Iteration 11: loss 18.26775360107422, pred 0.7259204387664795
Iteration 12: loss 16.186023712158203, pred 0.976814329624176
Iteration 13: loss 13.740848541259766, pred 1.2931349277496338
Iteration 14: loss 11.043739318847656, pred 1.6767877340316772
Iteration 15: loss 8.295835494995117, pred 2.119750738143921
Iteration 16: loss 5.75383138656616

Iteration 176: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 177: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 178: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 179: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 180: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 181: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 182: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 183: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 184: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 185: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 186: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 187: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 188: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 189: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 190: loss 2.2737367544323206e-13, pred 4.99999952316

Iteration 577: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 578: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 579: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 580: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 581: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 582: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 583: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 584: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 585: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 586: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 587: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 588: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 589: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 590: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 591: loss 2.2737367544323206e-13, pred 4.99999952316

Iteration 971: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 972: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 973: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 974: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 975: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 976: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 977: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 978: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 979: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 980: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 981: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 982: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 983: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 984: loss 2.2737367544323206e-13, pred 4.999999523162842
Iteration 985: loss 2.2737367544323206e-13, pred 4.99999952316

NameError: name 'model_path' is not defined